In [ ]:
!pip install nbformat --upgrade

In [1]:
import os
import json
import pandas as pd
import psycopg2
import matplotlib.pyplot as plt
import plotly.express as px
from psycopg2.extensions import ISOLATION_LEVEL_AUTOCOMMIT

In [23]:
connect = psycopg2.connect(
     host = "localhost",
     user = 'postgres',
     port = '5432',
     password = '01234',
     database = 'phonepy_transaction'
)

connect.set_isolation_level (ISOLATION_LEVEL_AUTOCOMMIT)

cursor = connect.cursor()

#Agg_insurance

cursor.execute ("select * from agg_insurance")
connect.commit()
tabel1 = cursor.fetchall()

Agg_insurance = pd.DataFrame(tabel1, columns=('State', 'Year', 'Quarter', 'Transaction_Name', 'Transaction_count', 'Transaction_amount'))


# Agg_transaction

cursor.execute ("select * from agg_transaction")
connect.commit()
tabel2 = cursor.fetchall()

Agg_Transaction = pd.DataFrame(tabel2, columns=('State', 'Year', 'Quarter', 'Transaction_type', 'Transaction_count', 'Transaction_amount'))


# Agg_User

cursor.execute ("select * from agg_user")
connect.commit()
tabel3 = cursor.fetchall()

Agg_User = pd.DataFrame(tabel3, columns=('State', 'Year', 'Quarter', 'Brand', 'Transaction_count', 'Percentage'))


# Map_insurance

cursor.execute ("select * from map_insurance")
connect.commit()
tabel4 = cursor.fetchall()

Map_insurance = pd.DataFrame(tabel4, columns=('State', 'Year', 'Quarter', 'District', 'Transaction_count', 'Transaction_amount'))


# Map_transaction

cursor.execute ("select * from map_transaction")
connect.commit()
tabel5 = cursor.fetchall()

Map_Transaction = pd.DataFrame(tabel5, columns=('State', 'Year', 'Quarter', 'District', 'Transaction_count', 'Transaction_amount'))


# Map_User

cursor.execute ("select * from map_user")
connect.commit()
tabel6 = cursor.fetchall()

Map_User = pd.DataFrame(tabel6, columns=('State', 'Year', 'Quarter', 'District', 'RegisteredUsers', 'AppOpens'))


# Top_insurance

cursor.execute ("select * from top_insurance")
connect.commit()
tabel7 = cursor.fetchall()

Top_insurance = pd.DataFrame(tabel7, columns=('State', 'Year', 'Quarter', 'Pincode', 'Transaction_count', 'Transaction_amount'))


#Top_Transaction

cursor.execute('select * from top_transaction')
connect.commit()
tabel8 = cursor.fetchall()

Top_Transaction = pd.DataFrame(tabel8, columns=('State', 'Year', 'Quarter', 'Pincode', 'Transaction_count', 'Transaction_amount'))


#Top_User

cursor.execute('select * from top_user')
connect.commit()
tabel9 = cursor.fetchall()

Top_User = pd.DataFrame(tabel9, columns=('State', 'Year', 'Quarter', 'Pincode', 'RegisteredUsers'))

In [54]:
def Transaction_amount_count_Y(year):
    #(Boolean Series)
    tacy = Agg_insurance[Agg_insurance['Year'] == year]
    tacy.reset_index(drop = True, inplace= True)

    tacyg = tacy.groupby('State')[['Transaction_count', 'Transaction_amount']].sum()
    tacyg.reset_index(inplace=True)

    fig_amount = px.bar(tacyg, x='State', y='Transaction_amount', title =f'{year} Transaction Amount', 
                         color_discrete_sequence= px.colors.sequential.Aggrnyl)
    fig_amount.show()

    fig_count = px.bar(tacyg,x = 'State', y='Transaction_count', title =f'{year} Transaction Count',
                       color_discrete_sequence= px.colors.sequential.Peach_r)
    fig_count.show()

In [55]:
Transaction_amount_count_Y(2022)

In [40]:
tacyg

,State,Transaction_count,Transaction_amount
0,Andaman & Nicobar,1349,1787385
1,Andhra Pradesh,103808,87005234
2,Arunachal Pradesh,930,1326049
3,Assam,19549,25315474
4,Bihar,67341,67646954
5,Chandigarh,1920,1854966
6,Chhattisgarh,20472,18231761
7,Dadra & Nagar Haveli & Daman & Diu,1616,1260309
8,Delhi,77044,72081600
9,Goa,5196,5979025
